# Computation for Dynamic Programming, Class 2

In class 1, we learned to do basic value function iteration (VFI) with even grid disretisation in economies with a discrete stochastic state. The aims of class 2 are simple:

1. Learn how to deploy recursive tools to tackle continuous stochastic state processes
2. Learn how to compute full recursive competitive equilibria, both by invoking the Welfare Theorems and through other means

Though the primary goal of this series is to learn to actually *do* the computation, a secondary goal is to solve meaningful models and illuminate the economic mechanisms at play. This class will consider a host of serious macro models; serious in their own right and in the history of the discipline. 

## Example 4: Lucas Asset Pricing Tree

The ideas introduced in lucas' asset pricing tree (*Asset Prices in an Exchange Economy*, 1978)
are both exhilarating and the perfect pedagogical next step. Lucas' tree will allow us to tackle continuous stochastic states. In addition, it will open a low-cost way to think rigorously about equilibrium asset prices.

The model is simple. Imagine there is one unit of an asset pricing "tree" which yields $z_t$ units of dividends (fruits) at time $t\in\{0,1,...\}$. $z_t >0$ is continuous and stochastic. There is a unit mass of consumers who trade shares of the tree. That is, the $i$ th individual brings $v_{i,t}$ units of shares into period $t$. The agent then chooses how many shares to hold, $v_{i,t+1}$, in that period to take to the following. Consumers have utility function $u(c)$ with the same properties as assumed in Class 1. The good consumed is the same good as that produced by the tree. Unlike the social planner, who can simply ignore prices, consumers take prices as given $p(z)$. It is a function of dividends as these fruits perfectly define the state of the aggregate economy. 

[Insert from SLP conditions needed for the value function to differentiable and envelope theorems to apply.]

In Bellman form, 

$$
V(v_i, z) = \max_{v_i'\in [0, (p(z)+z)v_i]} \bigg\{ u([p(z)+z]v_i - p(z)v_i')+\beta \mathbb{E}[V(v_i',z')|z]\bigg\} 
$$

To find the equilibrium condition of the problem, we can differentiate with respect to $v_i$ and $v_i'$ evaluated at the optimum (where $v_{i}'(z)$ is the policy function). Remembering the derivative with respect to the policy variable is zero evaluated at the optimum,

$$
\frac{dV(v_i,z)}{d v_i'} = - p(z) u_c(c) + \beta \mathbb{E}\bigg[\frac{dV(v_i',z')}{d v_i'}|z\bigg] = 0
$$

Rearranged, 

$$
p(z) u_c(c) = \beta \mathbb{E}\bigg[\frac{dV(v_i',z')}{d v_i'}|z\bigg]
$$

Now, differentiating with respect to the current state, 

$$
\frac{dV(v_i,z)}{d v_i} = (p(z)+z)u_c(c)
$$

Inserting this into the first expression but iterated forward a period, 

$$
p(z) u_c(c) = \beta \mathbb{E}\bigg[(p(z')+z')u_c(c')|z\bigg]
$$

From here we can recover the famous, and beautiful, equilibrium pricing condition,

$$
p(z)=  \mathbb{E}\bigg[\underbrace{\frac{\beta u_c(c')}{u_c(c)}}_{m_{t,t+1}} (p(z')+z')|z\bigg]
$$

$m_{t,t+1}$ is the much talked-about stochastic discount factor (SDF) which, really, is what makes asset pricing interesting. In the model, individuals adjust their plans to buy/sell shares to equate prices to make this equation hold. If we knew everything but $p(z)$, then this would simly be a functional equation on which we might apply recursive methods to approximate $p(z)$...

Luckily, under one further assumption we can learn about equilibrium prices in a low cost way. Suppose that everyone inherits the same number of shares in period 0: $v_{i,-1} = v_{-1}, \forall i \in [0,1]$. Realise that, in this case, $v_{i,0} = v_{0}, \forall i \in [0,1]$. This will also be true in every period! Since share purchases are identical across the economy, $c_{i,t} = c_{t}, \forall i \in [0,1]$. Consumption is constant across individuals. This will help us stay in the "representative agent" world. By definition (we will define many other RCEs more formally in this and future classes), markets clear in an RCE. The only is resource is $z_t$. Hence, market clearing implies that $c_t = z_t$ in every period. This is great news. Under equal inheritence, 

$$
p(z)=  \underbrace{\int_{z'}\frac{\beta u_c(z')}{u_c(z)}(p(z')+z')dF(z'|z)}_{T(p)}
$$

Now we will consider the dynamics of $z$. Suppose $\ln z' = \rho \ln z + \sigma \epsilon'$. Here, $\rho \in (0,1), \, \sigma > 0$ and $\epsilon'$ is drawn identically and independently from $N(0,1)$, which has a PDF $\phi$. This will help us because $z' = g(z, \epsilon') = z^\rho e^{\sigma \epsilon'}$.


As Lucas did, if we define helper functions $f(z):=p(z)u_c(z)$ and $h(z) := \beta \int_{\epsilon'} u_c(g(z,\epsilon'))g(z,\epsilon') \phi(\epsilon') d \epsilon'$, we can rewrite the pricing condition as:

$$
f(z) = \underbrace{h(z) + \int_{\epsilon'} f(g(z,\epsilon')) \phi(\epsilon') d\epsilon'}_{T(f)}
$$

Is this a contraction mapping? This [QuantEcon](https://python-advanced.quantecon.org/lucas_model.html) lesson includes the quick derivation showing it is. Now the question becomes, how can we do the iteration computationally?

Enter Tauchen discretisation. The idea is simple, and similar in spirit to the VFI we have already done. Approximate the continuous process using a discrete Markov chain. 

Split the grid into $N$ evenly spaced points, $z_i \in [z_0, ..., z_{N-1}], z_0 < z_1 < ... < z_{N-1}$. The pricing condition becomes:

$$
p(z_i)=  \sum_{j=0}^{N-1} {\frac{\beta u_c(z_j)}{u_c(z_i)}}(p(z_j)+z_j)P_{ij}, \, \forall z_i
$$

This looks similar to the discrete stochastic state in Class 1! To make further progress, we need decide on two things: 

1. How to pick the bounds of the dividend grid, $z_0$ and $z_{N-1}$?
2. How to build the probability matrix $\mathbf{P}$?


We want to approximated the dynamics of $z$. We know that, 

$$\ln d \sim N \bigg(0, \frac{\sigma^2}{1-\rho}\bigg)$$

If $l_i:=\ln z_i$, We pick bounds $l_0, l_{N-1}$ by choosing a multiple $m>0$ such that $l_0,l_{N-1} = -3m\frac{\sigma}{\sqrt{1-\rho^2}}, 3m\frac{\sigma}{\sqrt{1-\rho^2}}$. A standard choice is $m=3$, which captures 99.7% of observations.

So far so good. Building $\mathbf{P}$ is a bit trickier. We want $F(z'|z)$, the conditional CDF of $d'$. Remembering that $z' = z^\rho e^{\sigma \epsilon'}$, it is clear that:

$$\text{Pr}(z'< c|z) = \text{Pr}(\ln z' < \ln c|z) = \Phi\bigg(\frac{\ln c - \rho \ln z}{\sigma}\bigg)$$

So, $F(z'|z) = \Phi\bigg(\frac{\ln z' - \rho \ln z}{\sigma}\bigg)$. Since the points are evenly spaced, call half of the distance $\delta$. That is, $|z_i - z_j| = 2 \delta, \, j \in \{i-1, i+1\}$. Follow the following two-step procedure. First, for every entry in the matrix, compute:

$$ \tilde P_{ij} =  F(z_j+\delta|z_i) - F(z_j+\delta|z_i), \, j \in \{0,...,N-1\} $$

Since we have not spanned the whole number line, we need to scale up our probabilities so that the sum of each row is one. Compute the $\alpha$ such that, 

$$\sum_{j=0}^{N-1} (\tilde P_{ij} + \alpha_i) = 1 $$

Finally, define the matrix as:

$$P_{ij} = \tilde P_{ij}+\alpha_i$$

The second step helps us preserve the shape of the distribution as opposed to making the end bins abnormally large.

For a given utility function, we are ready to compute equilibrium prices!

In [23]:
# load base libraries for the class
import numpy as np
import matplotlib.pyplot as plt

# define a function which creates the z grid and transition matrix, P
def AR_1_tauchen(rho, sigma, N = 10, m = 3):
    """
    Takes an AR(1) process: ln z' = rho * ln z + sigma * epsilon 

    Returns:
    - z_vals: grid for z
    - P: transition matrix which approximate the process
    """
    # import propper libraries
    from scipy.stats import norm
    # upper bound for ln z
    l_upper = 3*m*sigma/np.sqrt(1-rho**2)
    # lower bound for ln z
    l_lower = - l_upper
    # create z grid
    z_grid = np.linspace(np.exp(l_lower), np.exp(l_upper), N)
    # delta 
    delta = (z_grid[1] - z_grid[0])/2
    # l grid
    l_grid = np.log(z_grid)
    # set up the grid for P tilde
    P_tilde = np.zeros((N,N))
    # fill in P tilde
    for i in range(N):
        for j in range(N):
            # normalised upper bound
            norm_upper = (np.log(z_grid[j]+delta)-rho*l_grid[i])/sigma
            # normalised lower bound (or zero)
            norm_lower = (np.log(max(1e-6, z_grid[j]-delta))-rho*l_grid[i])/sigma
            # calculate the prob of going to that bin
            P_tilde[i,j] = norm.cdf(norm_upper)-norm.cdf(norm_lower)
    # for every row compute the alpha and transform
    alphas = (1/N)*(1-np.sum(P_tilde, axis=1))
    # the final P
    P = P_tilde + alphas[:,np.newaxis]
    # return objects of interest
    return P, z_grid

# set up the class
class LucasAssetTree:
    # initialise 
    def __init__(self, grid_size, gamma , beta = 0.96, rho = 0.9, sigma = 0.25, m = 3, threshold = 1e-6):
        
        # set parameters
        self.N = grid_size
        self.gamma = gamma
        self.beta = beta
        self.rho = rho
        self.sigma = sigma
        self.m = m
        self.threshold = threshold

        # guess the risk-neutral, unconditional price
        self.p_initial = np.exp(sigma**2/(2*(1-rho**2)))/(1-beta)
    